In [154]:
import numpy as np
import pandas as pd

df = pd.read_csv('processedData.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,time,accX,accY,accZ,gyrX,gyrY,gyrZ,y,x
0,0,0,0.000000,-0.04,0.00,9.80,0.000000,-0.002182,0.000000,0.000965,0.0
1,1,1,0.021345,0.02,-0.10,9.70,0.000000,0.000000,0.000000,0.000440,0.0
2,2,2,0.041532,0.20,-0.14,9.65,0.025089,-0.010908,-0.004363,-0.000440,0.0
3,5,5,0.101133,-0.92,-0.61,9.94,0.001091,0.055632,0.027271,-0.004915,0.0
4,6,6,0.119408,0.29,-0.52,9.42,-0.061087,0.016362,0.035997,-0.006772,0.0


### Check for NaN

In [155]:
df.isnull().sum()

Unnamed: 0      0
Unnamed: 0.1    0
time            0
accX            1
accY            0
accZ            0
gyrX            0
gyrY            0
gyrZ            0
y               0
x               0
dtype: int64

### Get dx and dy and fill all NaN

In [156]:
#df["accX"] = df["accX"].cumsum()
#df["accY"] = df["accY"].cumsum()
df.loc[:,'dx'] = df['x'].diff().fillna(0.0) 
df.loc[:,'dy'] = df['y'].diff().fillna(0.0) 
df['accX'] = df.accX.ffill().add(df.accX.bfill()).div(2)
df['accY'] = df.accY.ffill().add(df.accY.bfill()).div(2)
df['accZ'] = df.accZ.ffill().add(df.accZ.bfill()).div(2)
df['gyrX'] = df.gyrX.ffill().add(df.gyrX.bfill()).div(2)
df['gyrY'] = df.gyrY.ffill().add(df.gyrY.bfill()).div(2)
df['gyrZ'] = df.gyrZ.ffill().add(df.gyrZ.bfill()).div(2)


### Split data to train and test sets

In [157]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
print("training examples:", len(train))
print("test:", len(test))

training examples: 153
test: 39


### Inputs and outputs

In [158]:
Xtrain = np.column_stack([train["accX"], train["accY"], train["accZ"], train["gyrX"], train["gyrY"], train["gyrZ"]]).T
Ytrain = np.column_stack([train["dx"], train["dy"]]).T

Xtest = np.column_stack([test["accX"], test["accY"], test["accZ"], test["gyrX"], test["gyrY"], test["gyrZ"]]).T
Ytest = np.column_stack([test["dx"], test["dy"]]).T
print("Shape of input: ", np.shape(Xtrain))
print("Shape of output: ", np.shape(Ytrain))

Shape of input:  (6, 153)
Shape of output:  (2, 153)


### Initialize parameters

In [159]:
def init_params(X, h, Y):
    np.random.seed(2)
    nX = np.shape(X)[0]
    nH = h
    nY = np.shape(Y)[0]
    W1 = np.random.randn(nH, nX)*0.01
    b1 = np.zeros((nH, 1))
    W2 = np.random.randn(nY, nH)*0.01
    b2 = np.zeros((nY, 1))
    W = [0., W1, W2]
    b = [0., b1, b2]
    return W, b

In [160]:
W, b = init_params(Xtrain, 6, Ytrain)


### RELU function

In [161]:
# Relu function
def ReLU(z):
    return np.maximum(0.,z)
# Relu derivative
def ReLUPrime(z):
    return (z > 0.)*1.

In [162]:
ReLUPrime(np.array([1.3,3.4,4.4,-0.4]))

array([1., 1., 1., 0.])

### Forward Propogation

In [163]:
def forward_prop(X, W, b):
    Z1 = W[1]@X + b[1]
    A1 = ReLU(Z1)
    Z2 = W[2]@A1 + b[2]
    A2 = Z2#ReLU(Z2)
    Z = [0., Z1, Z2]
    A = [0., A1, A2]
    return A, Z

In [164]:
W, b = init_params(Xtrain, 12, Ytrain)
A, Z = forward_prop(Xtrain, W, b)
print(A[1])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.05940034 0.04960543 0.05002678 ... 0.04526936 0.05016838 0.0553631 ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.0612967  0.04485646 0.04794124 ... 0.04156626 0.04676028 0.05396516]]


### Back prop

In [165]:
def backward_prop(A, Z, W, X, Y):
    m = np.shape(Y)[1]
    dZ2 = A[2] - Y
    dW2 = (1/m)*(dZ2@(A[1].T))
    #print(dW2)
    #print(dZ2,A[1].T)
    db2 = (1/m)*np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = (W[2].T @ dZ2) * ReLUPrime(Z[1])
    dW1 = (1/m)*(dZ1 @ (X.T))
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    dW = [0., dW1, dW2]
    db = [0., db1, db2]
    return dW, db

In [166]:
dW, db = backward_prop(A, Z, W, Xtrain, Ytrain)

### update parameters (gradient descent)

In [167]:
def update(W, b, dW, db, alpha):
    W[1] = W[1] - alpha*dW[1]
    b[1] = b[1] - alpha*db[1]
    W[2] = W[2] - alpha*dW[2]
    b[2] = b[2] - alpha*db[2]
    return W, b

### Cost function

In [168]:
def cost(A, Y):
    m = np.shape(Y)[1]
    J = -(1/(2*m))*np.sum((A[2] - Y)**2)
    return J

### Train model

In [169]:
def train_nn(X, h, Y, iterations):
    W, b = init_params(X, h, Y)
    for i in range(iterations):
        A, Z = forward_prop(X, W, b)
        dW, db = backward_prop(A, Z, W, X, Y)
        W, b = update(W, b, dW, db, alpha = 0.01)
        if i % 10 == 0:
            print("Cost " + " at ", i, cost(A, Y))
    return W, b



In [170]:
W, b = train_nn(Xtrain, h=100, Y = Ytrain, iterations=20000)

Cost  at  0 -4.952382000331479e-05
Cost  at  10 -3.3922686075768196e-05
Cost  at  20 -2.3314379005427968e-05
Cost  at  30 -1.609591089828571e-05
Cost  at  40 -1.118149538077151e-05
Cost  at  50 -7.834263294007681e-06
Cost  at  60 -5.553915879484575e-06
Cost  at  70 -4.000039626703556e-06
Cost  at  80 -2.940907684587129e-06
Cost  at  90 -2.218847307487326e-06
Cost  at  100 -1.7264939200742745e-06
Cost  at  110 -1.3906981099351181e-06
Cost  at  120 -1.1616486088704692e-06
Cost  at  130 -1.0054039020536819e-06
Cost  at  140 -8.988118119424135e-07
Cost  at  150 -8.260806912833812e-07
Cost  at  160 -7.76445898775573e-07
Cost  at  170 -7.425652226673015e-07
Cost  at  180 -7.194317181781985e-07
Cost  at  190 -7.036298225018801e-07
Cost  at  200 -6.928305400290313e-07
Cost  at  210 -6.854446552752441e-07
Cost  at  220 -6.803879772332732e-07
Cost  at  230 -6.769207940973008e-07
Cost  at  240 -6.745383852684388e-07
Cost  at  250 -6.728963456150071e-07
Cost  at  260 -6.717596485563183e-07
Cost  a

In [171]:
A, Z = forward_prop(Xtest, W, b)
J = cost(A, Ytest)
J
#W[1].shape

-6.689729979911258e-07